In [4]:
!pip install gdown
!gdown --fuzzy https://drive.google.com/file/d/16KFFwNgyA4kEC8q0ljGEWtNNzExWiN0M/view?usp=drive_link

Downloading...
From: https://drive.google.com/uc?id=16KFFwNgyA4kEC8q0ljGEWtNNzExWiN0M
To: /kaggle/working/rerank_rand_9.txt
100%|██████████████████████████████████████| 3.36M/3.36M [00:00<00:00, 46.8MB/s]


In [5]:
##### Đường dẫn đến file
import pandas as pd
import numpy as np

file_path = '/kaggle/working/rerank_rand_9.txt'
temp_file_path = '/kaggle/working/temp.txt'  # Tạm thời lưu lại dữ liệu mới

try:
    with open(file_path, 'r', encoding='utf-8') as file, open(temp_file_path, 'w', encoding='utf-8') as temp_file:
        for line in file:
            trimmed_line = line.split() #[:-30]  # Loại bỏ 10 từ cuối
            new_line = ' '.join(trimmed_line)  # Ghi lại dòng mới
            temp_file.write(new_line + '\n')
except FileNotFoundError:
    print("File không tìm thấy. Vui lòng kiểm tra đường dẫn.")
except Exception as e:
    print(f"Có lỗi xảy ra: {e}")

# Di chuyển file tạm thời sang file gốc
import shutil
shutil.move(temp_file_path, file_path)


'/kaggle/working/rerank_rand_9.txt'

In [6]:
corpus = pd.read_csv("/kaggle/input/fix-tokenize-corpus/corpus.csv")
cids = corpus['cid'].values

corpus.head()

,text,cid
0,"Thông_tư này hướng_dẫn tuần_tra , canh_gác bảo...",0
1,"1 . Hàng năm trước mùa mưa , lũ , Uỷ_ban_nhân_...",1
2,Tiêu_chuẩn của các thành_viên thuộc lực_lượng ...,2
3,"Nhiệm_vụ của lực_lượng tuần_tra , canh_gác đê ...",3
4,"Phù_hiệu của lực_lượng tuần_tra , canh_gác đê ...",4


In [7]:
def read_predict_file(file_path):
    predictions = {}
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            qid = parts[0]
            cids = [cid.split('_')[0] for cid in parts[1:]]
            predictions[qid] = cids
    return predictions

def parse_context(text):
    result = []
    parts = text.strip("[]").split("\n")
    for part in parts:
        part = part.strip()
        if "'" in part:
            result.append(part.strip("'"))
        else:
            result.extend(part.split())
    return result

# Đọc file CSV gốc (9456 sample cuối)
def read_ground_truth(train):
    # Lấy 9456 sample cuối
    df = train.head(9456)
    
    # Tạo từ điển để tra cứu nhanh
    ground_truth = {}
    for _, row in df.iterrows():
        ground_truth[str(row['qid'])] = row['cid']
    
    return ground_truth

# Tính MRR@10
def calculate_mrr(predictions, ground_truth, cids):
    mrr_scores = []
    error = []
    for qid, label_list in ground_truth.items(): 
        for ground_truth_cid in label_list[:]:
            if ground_truth_cid not in cids: label_list.remove(ground_truth_cid)
                
        if qid in predictions and label_list != []:
            predicted_cids = [str(cid) for cid in predictions[qid]]
            rank = None
            for i, cid in enumerate(predicted_cids):
                if cid in [str(label) for label in label_list]:
                    rank = 1/(i + 1)
                    break
            if rank != None:
                mrr_scores.append(rank)
            else:
                error.append(qid)
                mrr_scores.append(0)
    
    return error, np.mean(mrr_scores)

# Đường dẫn file
predict_file = '/kaggle/input/query-ela-jin/predict.zip' 
csv_file = '/kaggle/input/query-ela-jin/train_csv.csv'

In [8]:
train = pd.read_csv('/kaggle/input/split-data/test.csv')
train['cid'] = train['cid'].apply(lambda cid_value: list(map(int, cid_value.strip('[]').split())) if isinstance(cid_value, str) else [])
train['context'] = train['context'].apply(lambda x: parse_context(x))
ground_truth = read_ground_truth(train)
#ground_truth

In [9]:
predictions_rerank = read_predict_file("/kaggle/working/rerank_rand_9.txt")
error, mrr_score_rerank = calculate_mrr(predictions_rerank, ground_truth, cids)
print(f"MRR@10 rerank: {mrr_score_rerank:.4f}")

MRR@10 rerank: 0.7198
